In [0]:
%run "/Formula1/includes/configuration"

Out[15]: [FileInfo(path='dbfs:/mnt/formularacedata/presentation/', name='presentation/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/', name='processed/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/', name='raw/', size=0, modificationTime=0)]

In [0]:
%run "/Formula1/includes/common_functions"

- results_path = '/mnt/formularacedata/processed/results'
- driver_path = '/mnt/formularacedata/processed/drivers/'
- races_path = '/mnt/formularacedata/processed/races/'
- circuits_path = '/mnt/formularacedata/processed/circuits/'
- constructors_path = '/mnt/formularacedata/processed/constructors/'

In [0]:
results_df = spark.read.format('delta').load(results_path).withColumnRenamed('number','result_number').withColumnRenamed("race_id","results_race_id").withColumnRenamed("time","race_time")
results_df.show(3)

+---------+---------------+---------+--------------+-------------+----+--------+-------------+--------------+------+----+---------+------------+-----------+----+----------------+-----------------+--------------------+
|result_id|results_race_id|driver_id|constructor_id|result_number|grid|position|position_text|position_order|points|laps|race_time|milliseconds|fastest_lap|rank|fastest_lap_time|fastest_lap_speed|      ingestion_date|
+---------+---------------+---------+--------------+-------------+----+--------+-------------+--------------+------+----+---------+------------+-----------+----+----------------+-----------------+--------------------+
|     7573|              1|        1|             1|            1|  18|    null|            D|            20|   0.0|  58|       \N|        null|         39|  13|        1:29.020|              214|2023-06-29 19:38:...|
|     7563|              1|        2|             2|            6|   9|      10|           10|            10|   0.0|  58|   +7.0

In [0]:
driver_df = spark.read.format('delta').load(driver_path).withColumnRenamed("name","driver_name").withColumnRenamed("number","driver_number").withColumnRenamed("nationality","driver_nationality")


In [0]:
from pyspark.sql.functions import to_date,col

races_df = spark.read.format('delta').load(races_path).withColumnRenamed("name","race_name").withColumn("race_date",to_date(col("race_timestamp")))

In [0]:
circuits_df = spark.read.format('delta').load(circuits_path).withColumnRenamed('location','circuit_location')

In [0]:
constructors_df = spark.read.format('delta').load(constructors_path).withColumnRenamed("name","team")

In [0]:
from pyspark.sql.functions import current_timestamp

df = results_df.join(races_df,results_df.results_race_id == races_df.race_id,"inner") \
               .join(driver_df,results_df.driver_id == driver_df.driver_id,"inner") \
                .join (constructors_df,results_df.constructor_id == constructors_df.constructor_id, "inner") \
                .join (circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner") \
                .withColumn("created_date",current_timestamp())

race_results_df = df.select("race_id","race_year","race_name","race_date","circuit_location","driver_name","driver_number","driver_nationality","team","grid","fastest_lap","race_time","points","position","created_date").orderBy(df.race_year.desc(),df.race_name,df.points.desc())

In [0]:
from pyspark.sql.functions import col,expr

race_results_df.filter(expr("race_name = 'Abu Dhabi Grand Prix' and race_year = 2020")).orderBy(expr("points").desc()).show(3)

+-------+---------+--------------------+----------+----------------+---------------+-------------+------------------+--------+----+-----------+-----------+------+--------+--------------------+
|race_id|race_year|           race_name| race_date|circuit_location|    driver_name|driver_number|driver_nationality|    team|grid|fastest_lap|  race_time|points|position|        created_date|
+-------+---------+--------------------+----------+----------------+---------------+-------------+------------------+--------+----+-----------+-----------+------+--------+--------------------+
|   1047|     2020|Abu Dhabi Grand Prix|2020-12-13|       Abu Dhabi| Max Verstappen|           33|             Dutch|Red Bull|   1|         14|1:36:28.645|  25.0|       1|2023-06-29 21:14:...|
|   1047|     2020|Abu Dhabi Grand Prix|2020-12-13|       Abu Dhabi|Valtteri Bottas|           77|           Finnish|Mercedes|   2|         40|    +15.976|  18.0|       2|2023-06-29 21:14:...|
|   1047|     2020|Abu Dhabi Grand 

In [0]:
from pyspark.sql.functions import col,expr

race_results_df.filter(expr("race_date is not null and driver_number is not null")).count()

Out[33]: 4465

In [0]:
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_id = src.race_id"
merge_delta_data(race_results_df, 'f1_presentation', 'race_results', presentation_folder_path, merge_condition, 'race_id')